In [1]:
%%bash
nvidia-smi

Tue Nov 14 14:23:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:06.0 Off |                    0 |
| N/A   62C    P0              30W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd

conv_dataframe = pd.read_csv("../data/small-conversations.csv", sep="\t")
conv_dataframe

,conversations,tensor_size
0,Below is an instruction that describes a task....,103
1,Below is an instruction that describes a task....,199
2,Below is an instruction that describes a task....,156
3,Below is an instruction that describes a task....,137
4,Below is an instruction that describes a task....,189
...,...,...
255,Below is an instruction that describes a task....,154
256,Below is an instruction that describes a task....,163
257,Below is an instruction that describes a task....,54
258,Below is an instruction that describes a task....,82


In [3]:
from datasets import load_dataset
from datasets import Dataset
#conv_dataframe = conv_dataframe[:900]
dataset = Dataset.from_pandas(conv_dataframe).train_test_split(test_size=0.2, seed=42)

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import re
import sys
import torch
from transformers import (LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig, AutoModelForCausalLM,
    AutoTokenizer)

model_path = 'openlm-research/open_llama_3b_v2'

model = LlamaForCausalLM.from_pretrained(
    model_path, 
    load_in_8bit=True,
    device_map='auto', 
    # If passing a string for `device_map`, please choose 'auto', 'balanced', 'balanced_low_0' or 'sequential'.
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = LlamaTokenizer.from_pretrained(model_path)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model_modules = str(model.modules)
print(model_modules)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


<bound method Module.modules of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear8bitLt(in_features=8640, out_features=3200, bias=False)
          (up_proj): Linear8bitLt(in_features=3200, out_features=8640, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_at

In [5]:
tokenizer, tokenizer.vocab_size

(LlamaTokenizer(name_or_path='openlm-research/open_llama_3b_v2', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),
 32000)

In [6]:
import re
model_modules = str(model.modules)
pattern = r'\((\w+)\): Linear'
linear_layer_names = re.findall(pattern, model_modules)

names = []
# Print the names of the Linear layers
for name in linear_layer_names:
    names.append(name)
target_modules = list(set(names))
target_modules

['gate_proj',
 'k_proj',
 'lm_head',
 'o_proj',
 'q_proj',
 'v_proj',
 'up_proj',
 'down_proj']

In [7]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch
from transformers.trainer_callback import TrainerCallback
import os
import sys
import time
from transformers import BitsAndBytesConfig
from trl import SFTTrainer
from transformers import TrainingArguments


#target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head'] # all linear layers
#or
#target_modules = ['q_proj','v_proj', 'k_proj', 'o_proj'] # attention layers


lora_config = LoraConfig(
    r=16, #r=16
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules=target_modules,
    task_type="CAUSAL_LM",
)

print("Extracting parameter efficient model ...")
s_time = time.time()
refined_model = get_peft_model(model, lora_config)
e_time = time.time()
refined_model.print_trainable_parameters()
print("PEFT loading time: {} seconds".format(e_time - s_time))

base_dir = "llama"

per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear" 

print("Setting up Training arguments ...")

training_args = TrainingArguments(
    output_dir=base_dir,
    save_strategy="epoch",
    evaluation_strategy="steps",
    num_train_epochs = 5.0,
    logging_strategy="epoch",
    logging_steps=25,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    weight_decay=0.001,
    learning_rate=learning_rate,
    fp16=False, #True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    max_steps=-1,
    report_to="tensorboard"
)

print("Setting up SFTTrainer ...")

s_time = time.time()
trainer = SFTTrainer(
    refined_model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="conversations",
    args=training_args,
    max_seq_length=200,
    tokenizer=tokenizer,
    packing=False,
)

e_time = time.time()
print("SFTTTrainer setting up time: {} seconds".format(e_time - s_time))

print("Start training ...")
trainer.train()
print("Finished training ...")

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Extracting parameter efficient model ...
trainable params: 25,989,120 || all params: 3,452,462,720 || trainable%: 0.7527704745208661
PEFT loading time: 0.9485962390899658 seconds
Setting up Training arguments ...
Setting up SFTTrainer ...


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [00:00<00:00, 494.57 examples/s]
/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SFTTTrainer setting up time: 0.4101696014404297 seconds
Start training ...


Step,Training Loss,Validation Loss
25,3.190400,3.044922
50,3.101000,2.998047


Finished training ...


In [8]:
'''from tensorboard import notebook
log_dir = "llama/runs"
notebook.start("--logdir {} --port 4001".format(log_dir))'''

'from tensorboard import notebook\nlog_dir = "llama/runs"\nnotebook.start("--logdir {} --port 4001".format(log_dir))'

In [9]:
# test fine-tuned model
# Number 1000, 1001 from data.json

#test_strings = ["Hello!\nI am running a local instance of Galaxy (build 22.05). I installed the latest version of Deepvariant (1.4.0+galaxy0) which installed without any errors. However, when I try to run Deepvariant on BAM files output from HISAT2, the error “Fatal error: Exit code 127 ()” comes up. Further, it says that the tool generated the following error: “line 9: run_deepvariant: command not found”.\nWhen I look at the backend to see what process Galaxy is going through, even after installation of the tool, the following line keeps repeating on the command line interface:\nuvicorn.access INFO 2022-12-22 13:43:26,629 [pN:main.1,p:100965,tN:MainThread] 127.0.0.1:58158 - “GET /api/tool_shed_repositories?name=deepvariant&owner=iuc HTTP/1.1” 200\nI don’t understand this error. Could someone please help me out?\nI am running the same job on Galaxy.eu server and it is running (for a few hours now) but in the local instance in errors out pretty much instantly.\nThanks!",
#"Dear Sir,\nKindly help in this regards I was trying to make a de novo contig using trinity and it is running since from one week.\nIs it ok??? or did I out something wrong\nKindly help"
#               ]
# 

test_strings = [
    "{ “code_desc”: “”, “desc”: “Fatal error: Exit code 126 ()”, “error_level”: 3, “exit_code”: 126, “type”: “exit_code” }  Is there a list of exit codes somewhere so I can determine what went wrong?",
    "Hoping to revive this. Same problem here. Ran Trim Galore on approx 50fastq files. 44 of them worked fine (produced output files). 3 of the paired files did not. So Sample1 R1 Sample1 R2 Sample2 R1 Sample2 R2 etc…did not work. The error for these say: Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>> The files look like they have the same data in header as the files that worked. Please help!",  
]

#response_test_strings = [
#"Hello, I can’t give a full answer but I can maybe guide you in the right direction and maybe someone that can give a better answer will reply.\nGiven the error it looks like deepvariant is not installed (not found). The tool is using a “docker tool dependency”, in other words it needs a container where deepvariant is installed. If you have not checked this yet then I think this is the place to start. Below two links where you may find some more information.\nhttps://docs.galaxyproject.org/en/master/admin/special_topics/mulled_containers.html\n  \n      \n\n      training.galaxyproject.org\n  \n\n  \n    \n\nGalaxy Training: Tool Dependencies and Containers\n\n  Galaxy is an open-source project. Everyone can contribute...\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nThe requirement can be seen here:\n  \n\n      github.com\n  \n\n  \n    galaxyproject/tools-iuc/blob/master/tools/deepvariant/macros.xml\n\n\n      <macros>\n    <token name="@TOOL_VERSION@">1.4.0</token>\n    <token name="@SUFFIX_VERSION@">0</token>\n    <xml name="edam_ontology">\n        <edam_topics>                                                                                  \n            <edam_topic>topic_0199</edam_topic>\n        </edam_topics>\n        <edam_operations>\n            <edam_operation>operation_3227</edam_operation>\n        </edam_operations>\n    </xml>\n    <xml name="requirements">\n        <requirements>\n            <container type="docker">google/deepvariant:@TOOL_VERSION@</container>\n        </requirements>\n    </xml>\n    <xml name="citations">\n        <citations>\n            <citation type="doi">10.1038/nbt.4235</citation>\n        </citations>\n\n\n\n\n  This file has been truncated. show original\n\n  \n\n  \n    \n    \n  \n\n  \n\n", 
# "Hello @Sachin_Srivastava\nIf the job is running (yellow/peach dataset), it is usually best to allow it to run. The same is true for queued jobs (grey dataset). This applies to jobs (any tool) executed at a public Galaxy server.\n20 GB of fastq data – uncompressed – creates a very large assembly job. If it fails later on for exceeding resources (red dataset), you’ll need to do one or more of these:\n\nTry a rerun to eliminate cluster issues\nMore QA/QC on the input reads (always recommended)\nConsider downsampling the reads (tool: Seqtk)\nPossibly need to move to your own Galaxy server where more resources can be allocated. The GVL version of Cloudman is one option: https://launch.usegalaxy.org/catalog\n\n\nI added some tags to your post that will find prior Q&A about the above actions. Or, you can search the forum with those keywords (not all posts get tagged).\nYou didn’t state where you are working. But, if by chance at Galaxy Main https://usegalaxy.org, I can let you know that the cluster that runs Trinity (and Unicycler + RNA-Star) is very busy. Longer queue times are expected. If you delete the current job and rerun, that will only place your job back at the end of the queue again, extending wait time.\nThanks!"]

predictions = []

for test in test_strings:
    #prompt = """<s> [INST] {} [/INST]""".format(test)
    prompt= """Below is an instruction that describes a task. Write a response that appropriately completes the request.
    
    ### Instruction:
    
    {}
    
    ### Response: \n""".format(test)
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda')

    outputs = refined_model.generate(input_ids=input_ids, max_new_tokens=256)
    predictions.append(tokenizer.decode(outputs[0]))

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [10]:
def extract_response_text(input_string):
    start_marker = '### Response: \n'
    end_marker = '###'
    start_index = input_string.find(start_marker)
    if start_index == -1:
        return None
    
    start_index += len(start_marker)
    
    end_index = input_string.find(end_marker, start_index)
    if end_index == -1:
        return input_string[start_index:]
    
    return input_string[start_index:end_index].strip()
    
for i in range(len(test_strings)): 
    pred = predictions[i]
    text = test_strings[i]
    print(text+'\n')
    print(pred+'\n')
    print(extract_response_text(pred))
    print("==============================")

{ “code_desc”: “”, “desc”: “Fatal error: Exit code 126 ()”, “error_level”: 3, “exit_code”: 126, “type”: “exit_code” }  Is there a list of exit codes somewhere so I can determine what went wrong?

<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
    
    ### Instruction:
    
    { “code_desc”: “”, “desc”: “Fatal error: Exit code 126 ()”, “error_level”: 3, “exit_code”: 126, “type”: “exit_code” }  Is there a list of exit codes somewhere so I can determine what went wrong?
    
    ### Response: 
    
    { “code_desc”: “”, “desc”: “Fatal error: Exit code 126 ()”, “error_level”: 3, “exit_code”: 126, “type”: “exit_code” }  No, there is not a list of exit codes.
    
    ### Response: 
    
    { “code_desc”: “”, “desc”: “Fatal error: Exit code 126 ()”, “error_level”: 3, “exit_code”: 126, “type”: “exit_code” }  No, there is not a list of exit codes.
    
    ### Response: 
    
    { “code_desc”: “”, “desc”: “Fatal error: Exit code

In [ ]:
# Reload PEFT fined tuned model
# https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo
'''from transformers import LlamaTokenizer, LlamaForCausalLM

saved_model_path = 'llama/checkpoint-2'
re_tokenizer = LlamaTokenizer.from_pretrained(saved_model_path)
re_model = LlamaForCausalLM.from_pretrained(saved_model_path, device_map='auto')'''

In [ ]:

'''# test fine-tuned model
# Number 1000, 1001 from data.json

#re_tokenizer.pad_token = "[PAD]"
re_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
re_tokenizer.padding_side = "left"

test_strings = ["Hello!\nI am running a local instance of Galaxy (build 22.05). I installed the latest version of Deepvariant (1.4.0+galaxy0) which installed without any errors. However, when I try to run Deepvariant on BAM files output from HISAT2, the error “Fatal error: Exit code 127 ()” comes up. Further, it says that the tool generated the following error: “line 9: run_deepvariant: command not found”.\nWhen I look at the backend to see what process Galaxy is going through, even after installation of the tool, the following line keeps repeating on the command line interface:\nuvicorn.access INFO 2022-12-22 13:43:26,629 [pN:main.1,p:100965,tN:MainThread] 127.0.0.1:58158 - “GET /api/tool_shed_repositories?name=deepvariant&owner=iuc HTTP/1.1” 200\nI don’t understand this error. Could someone please help me out?\nI am running the same job on Galaxy.eu server and it is running (for a few hours now) but in the local instance in errors out pretty much instantly.\nThanks!",
"Dear Sir,\nKindly help in this regards I was trying to make a de novo contig using trinity and it is running since from one week.\nIs it ok??? or did I out something wrong\nKindly help"
               ]

#response_test_strings = [
#"Hello, I can’t give a full answer but I can maybe guide you in the right direction and maybe someone that can give a better answer will reply.\nGiven the error it looks like deepvariant is not installed (not found). The tool is using a “docker tool dependency”, in other words it needs a container where deepvariant is installed. If you have not checked this yet then I think this is the place to start. Below two links where you may find some more information.\nhttps://docs.galaxyproject.org/en/master/admin/special_topics/mulled_containers.html\n  \n      \n\n      training.galaxyproject.org\n  \n\n  \n    \n\nGalaxy Training: Tool Dependencies and Containers\n\n  Galaxy is an open-source project. Everyone can contribute...\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nThe requirement can be seen here:\n  \n\n      github.com\n  \n\n  \n    galaxyproject/tools-iuc/blob/master/tools/deepvariant/macros.xml\n\n\n      <macros>\n    <token name="@TOOL_VERSION@">1.4.0</token>\n    <token name="@SUFFIX_VERSION@">0</token>\n    <xml name="edam_ontology">\n        <edam_topics>                                                                                  \n            <edam_topic>topic_0199</edam_topic>\n        </edam_topics>\n        <edam_operations>\n            <edam_operation>operation_3227</edam_operation>\n        </edam_operations>\n    </xml>\n    <xml name="requirements">\n        <requirements>\n            <container type="docker">google/deepvariant:@TOOL_VERSION@</container>\n        </requirements>\n    </xml>\n    <xml name="citations">\n        <citations>\n            <citation type="doi">10.1038/nbt.4235</citation>\n        </citations>\n\n\n\n\n  This file has been truncated. show original\n\n  \n\n  \n    \n    \n  \n\n  \n\n", 
# "Hello @Sachin_Srivastava\nIf the job is running (yellow/peach dataset), it is usually best to allow it to run. The same is true for queued jobs (grey dataset). This applies to jobs (any tool) executed at a public Galaxy server.\n20 GB of fastq data – uncompressed – creates a very large assembly job. If it fails later on for exceeding resources (red dataset), you’ll need to do one or more of these:\n\nTry a rerun to eliminate cluster issues\nMore QA/QC on the input reads (always recommended)\nConsider downsampling the reads (tool: Seqtk)\nPossibly need to move to your own Galaxy server where more resources can be allocated. The GVL version of Cloudman is one option: https://launch.usegalaxy.org/catalog\n\n\nI added some tags to your post that will find prior Q&A about the above actions. Or, you can search the forum with those keywords (not all posts get tagged).\nYou didn’t state where you are working. But, if by chance at Galaxy Main https://usegalaxy.org, I can let you know that the cluster that runs Trinity (and Unicycler + RNA-Star) is very busy. Longer queue times are expected. If you delete the current job and rerun, that will only place your job back at the end of the queue again, extending wait time.\nThanks!"]

predictions = []

for test in test_strings:
  prompt = """ Below is a question asked by a user: \n
  ### Instruction:
  {}. 
  ### Response by the system to the above instruction:""".format(test)

  input_ids = re_tokenizer.encode(prompt, return_tensors="pt").to('cuda')

  outputs = re_model.generate(input_ids=input_ids, 
                           max_new_tokens=156, 
                           do_sample=True, 
                           top_p=0.9,
                           temperature=0.9
                          )
  predictions.append(tokenizer.decode(outputs[0]))
    

def extract_response_text(input_string):
    start_marker = 'Response by the system to the above instruction:'
    end_marker = '###'
    start_index = input_string.find(start_marker)
    if start_index == -1:
        return None
    
    start_index += len(start_marker)
    
    end_index = input_string.find(end_marker, start_index)
    if end_index == -1:
        return input_string[start_index:]
    
    return input_string[start_index:end_index].strip()

for i in range(len(test_strings)): 
  pred = predictions[i]
  text = test_strings[i]
  print(text+'\n')
  print(pred+'\n')
  print(extract_response_text(pred))
  print("==============================")'''